In [ ]:
%pip install selenium
%pip install pandas
%pip install pandas

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tqdm.notebook import tqdm
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [3]:
# 데이터 불러오기
name =pd.read_csv("series_list.csv")

In [4]:
# 검색할 영화 추출 
inp =name["title"]
#추출할 영화 개수 선택 
inp2 = inp[0:10]
inp2

0              두 여인
1                이창
2                계모
3                롤라
4    2001 스페이스 오디세이
5               수색자
6         티파니에서 아침을
7             악마의 씨
8            거미집의 성
9                휴일
Name: title, dtype: object

In [6]:
def is_valid_float(element: str) -> bool:
    try:
        float(element)
        return True
    except ValueError:
        return False

In [2]:
chrome_options = Options()
chrome_options.add_argument('--window-size=1920,1080')
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

driver = webdriver.Chrome()
driver.get("https://pedia.watcha.com/ko-KR")
driver.implicitly_wait(4)
#크롤링 결과를 데이터 프레임으로 저장 
result = pd.DataFrame(data=None, columns=("title",'img_url','year','channel','genre','country','rating', "star", 'smry',"review"))
for title in tqdm(inp2):
    driver.get("https://pedia.watcha.com/ko-KR")
    #제목 검색
    elem = driver.find_element(By.ID, 'desktop-search-field')
    elem.send_keys(title)
    elem.send_keys(Keys.RETURN)
    elem.clear()
    driver.implicitly_wait(4)
    b =[]

    # 변수 선언 
    img_url = ''
    year = ''
    genre = ''
    point = ''
    smry =''



    try:
        check = 0
        if driver.find_element(By.XPATH, '//*[@id="root"]/div/div[1]/section/section/div[3]/div[1]/section/section[2]/header').text=='시리즈':
            series_list = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[1]/section/section/div[3]/div[1]/section/section[3]/div/div[1]/div/ul/li')
            for i in range(1, len(movie_list)+1):
                series = driver.find_element(By.CSS_SELECTOR,'#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > section > div.css-1x6fzl1.e1szkzar0 > div.css-13o7eu2.e1pww8ij0 > section > section:nth-child(2) > div > div.css-a50hm3.e1yzo30x5 > div > ul > li:nth-child(%d) > a > div.css-1pkjqps.e1ic68ft0 > div.css-m5a6g.e1ic68ft1 > div.css-9jne0z.e1gexyxq4'%(i))
                if title== movie.text:
                    check = 1    
                    movie.click()
                    break;  
        elif driver.find_element(By.CSS_SELECTOR, '//*[@id="root"]/div/div[1]/section/section/div[3]/div[1]/section/section[3]/header').text=='시리즈':
            movie_list = driver.find_elements(By.CSS_SELECTOR, '#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > section > div.css-1x6fzl1.e1szkzar0 > div.css-13o7eu2.e1pww8ij0 > section > section:nth-child(2) > div > div.css-a50hm3.e1yzo30x5 > div > ul > li')
            for i in range(1, len(movie_list)+1):
                movie = driver.find_element(By.CSS_SELECTOR,'#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > section > div.css-1x6fzl1.e1szkzar0 > div.css-13o7eu2.e1pww8ij0 > section > section:nth-child(2) > div > div.css-a50hm3.e1yzo30x5 > div > ul > li:nth-child(%d) > a > div.css-1pkjqps.e1ic68ft0 > div.css-m5a6g.e1ic68ft1 > div.css-9jne0z.e1gexyxq4'%(i))
                if title== movie.text:
                    check = 1    
                    movie.click()
                    break; 
        else:
            continue
                   
        if check == 0:
            continue

        # 리뷰 url 저장  
        Url=driver.current_url+"/comments"

        #장르 연도 크롤링
        try:
            info_list = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[1]/div[4]/div/div[2]').text
            info_list = info_list.split(' · ')
            year = info_list[0]
            channel = info_list[1]
            genre = info_list[2]
            info_list2 = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[1]/div[4]/div/div[3]').text
            info_list2 = info_list2.split(' · ')
            country = info_list2[0]
            rating = info_list2[1]
        except NoSuchElementException:
            pass
        
        #img url 저장 
        try:
            img = driver.find_element(By.XPATH,'//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[2]/section/div[1]/div/div/img')
            img_url = img.get_attribute('src')
        except NoSuchElementException:
            pass


        #줄거리 요약, 만약 줄거리가 없을 경우 빈 내용 저장 
        try:  
            smry = driver.find_element(By.CSS_SELECTOR, '#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > div > div.css-d3kywm.e11kyjwk3 > div > div.css-1oak90x.e11kyjwk1 > div > div.css-2tqido.e1yew2869 > section.css-3btvnx.e1yew2868 > div.css-g8juaf.e1yew2865 > section.css-291cts.e1yew2864').text
        except NoSuchElementException:
            pass

        # 별점 저장 
        try:
            point = driver.find_element(By.CSS_SELECTOR, '#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > div > div.css-d3kywm.e11kyjwk3 > div > div.css-1oak90x.e11kyjwk1 > div > div.css-2tqido.e1yew2869 > section > div.css-7ckxl2.e1yew2867 > section > span').text
            point = point.strip('평균 ★')
            point = point.split("(")
            star = point[0]
        except NoSuchElementException:
            pass

        driver.get(Url)
        time.sleep(0.5)
        #스크롤
        driver.execute_script("window.scrollTo(0, 2000)")
        time.sleep(0.5)
        driver.execute_script("window.scrollTo(2000, 4000)")
        time.sleep(0.5)
        driver.execute_script("window.scrollTo(4000, 6000)")
        time.sleep(0.5)
    
        # 리뷰 및 평점 크롤링 
        x = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[1]/section/section/div/div/div/ul/div')
        for i in range(0,len(x)-2) : 
            a = driver.find_element(By.CSS_SELECTOR,'#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > section > div > div > div > ul > div:nth-child(%d) > div.css-2occzs.egj9y8a1'%(i+1)).text
            a = a.replace('\n','')
            a = a.replace('#','')
            b.append(a)

        # 데이터 추가 
        new = pd.DataFrame({"title",'img_url','year','channel','genre','country','rating', "star", 'smry',"review"})
        result = pd.concat([result, new], ignore_index=True)
    except NoSuchElementException:
        pass

result.to_csv('review.csv', index =False, encoding="UTF-8-sig")

driver.close()

NameError: name 'inp2' is not defined

In [9]:
result

,title,img_url,year,genre,country,star,smry,review
0,두 여인,https://an2-img.amz.wtchn.net/image/v2/ZVZqji-...,1960,드라마/전쟁,"이탈리아, 프랑스",3.4,남편을 잃은 체시라는 신앙심 깊은 딸 로제타를 홀로 키운다. 2차 대전 중 연합군의...,"[도망칠 수 없는 곳, 참혹하지만 견딜 수 밖에 없었던 그녀들정말 개같은 세상이야...."
1,이창,https://an2-img.amz.wtchn.net/image/v2/_wbZtEj...,1954,미스터리/로맨스/스릴러,미국,4.0,사진 작가인 제프리스는 촬영 도중 다리가 부러져 휠체어에서 꼼짝할 수 없는 처지이다...,"[이름이 친근하다 맘에든다, <이창>은 군더더기 없이 완벽하게 조립되어 만들어진 영..."
2,계모,,1967,드라마,한국,4.0,초혼에 실패한 그녀는 모 재벌의 후처로 들어가서 열성을 다해 가정을 이끌어 간다. ...,[]
3,롤라,https://an2-img.amz.wtchn.net/image/v2/Uj0mt8s...,1961,드라마/로맨스,"프랑스, 이탈리아",3.5,"프랑스 낭트의 항구, 카바레 댄서인 롤라(아누크 에메 분)는 7년 전에 떠난 연인 ...",[쉘부르의 우산 보석상 롤랑 카사르의 아름다운 첫사랑 회상.먼저 감상한 3편(로슈포...
4,2001 스페이스 오디세이,https://an2-img.amz.wtchn.net/image/v2/jOf9ycx...,1968,SF/모험,"영국, 미국",4.0,인류에게 문명의 지혜를 가르쳐 준 검은 돌기둥의 정체를 밝히기 위해서 목성으로 향하...,"[볼때마다 경이롭지만 졸린건 참을 수 없다..., 자그마치 68년이다. 지금봐도 사..."
5,수색자,https://an2-img.amz.wtchn.net/image/v2/AsEKbPU...,2021,스릴러,한국,1.8,억울하게 죽은 영혼들의 무덤 DMZ\n\n어두운 밤 총성이 울린 후 파견 나온 교육...,"[이건 아닌데...싶은 부분을 세다가포기했다초반에, 군비리가 없어지는 세상을 기대하..."
6,티파니에서 아침을,https://an2-img.amz.wtchn.net/image/v2/cO4jaN5...,1961,코미디/로맨스/드라마,미국,3.8,"1940년대초 뉴욕, 검은 선글라스에 화려한 장신구로 치장한 한 여성이 택시에서 내...","[파격적인 캐릭터조차 달콤한 오드리 헵번., 사람은 누구나 자기만의 쇼윈도가 있다...."
7,악마의 씨,https://an2-img.amz.wtchn.net/image/v2/W54jfbR...,1968,공포/드라마/스릴러,미국,3.8,로즈 마리는 배우인 남편과 맨하탄의 공동식 아파트에 이주하면서 기괴한 일에 빠져든다...,[무력감의 공포. 다 알면서도 어쩔 수도 없는 로즈마리와 관객.Pray to par...
8,휴일,https://an2-img.amz.wtchn.net/image/v2/2J3D97d...,1968,드라마,한국,3.8,"어느 일요일, 빈털터리 청년 허욱은 사랑하는 지연을 만나러 간다. 지연은 낙태 수술...",[20세기의 한국 불황기 시절에 나온 문학 소설의 클리셰 요소들이 깔려있어 진부하게...
